# Integrating AWS API Gateway with AWS Kinesis

## Introduction

In this lesson you will learn to create and configure a REST API with an Amazon Kinesis proxy integration. We will build this integration, on the previously created REST API with Kafka REST proxy integration (see AWS API Gateway lesson).

## Create an IAM role for API access to Kinesis
    
To allow the API to invoke Kinesis actions, you must have appropriate IAM policies attached to an IAM role. To enable full access to Kinesis, you can create an IAM role in the IAM console that assumes the **AmazonKinesisFullAccessRole** policy. This will enable both read-write actions in Kinesis.
    
Make sure API Gateway is a trusted entity of the role and has assumed the execution role **sts:AssumRole**. The trust relationships of this role should look like the one below:

<p align="center">
    <img src="images/Kinesis Role.png" width="800" height="300"/>
</p>

## List streams in Kinesis
To begin building our integration navigate to the **Resources** tab of the previously created API. Open the **Create Resource** in **Actions**

<p align="center">
    <img src="images/Create Resources.png" width="800" height="250"/>
</p>

Under **Resource Name**, type `streams` (in the photo example provided `test-streams` is used instead).

<p align="center">
    <img src="images/Streams Resource.png" width="650" height="300"/>
</p>

Leave the rest as default and select **Create Resource**.

Choose the created **streams** resource, from Actions select **Create Method** and select `GET` from the drop-down list and click on the tick next to it.

<p align="center">
    <img src="images/List Streams.png" width="600" height="300"/>
</p>

In the **Setup** panel you will need to select the following
 - For **Integration type** select **AWS Service**
- For **AWS Region** choose *us-east-1*
- For **AWS Service** select **Kinesis**,
- For **HTTP method** select `POST` (as we will have to invoke Kinesis's `ListStreams` action)
- For **Action Type** select **User action name**
- For **Action** type `ListStreams`
- For **Execution role** you should copy the ARN of your Kinesis Access Role (created in the previous section).

<p align="center">
    <img src="images/List Stream Setup.png" width="650" height=400"/>
</p>


Finally, press **Save**. This will prompt you to the **Method Execution**.

<p align="center">
    <img src="images/List Streams Method Execution.png" width="600" height="400"/>
</p>

From here select the **Integration Request** panel. Expand the HTTP Header panel and select the following options:
- Choose **Add header** button
- Under **Name** type **Content-Type**
- Under **Mapped form** type **x-amz-json-1.1**
- Press the Create tick button

<p align="center">
    <img src="images/List Streams HTTP Headers.png" width="600" height="400"/>
</p>
   
Expand the **Mapping Templates** panel and select the following options:
- Choose **Add mapping template** button
- Under **Cotent-Type** type **application/json**
- Press the Create tick button and choose Yes, secure this integration

<p align="center">
    <img src="images/List Streams Mapping Templates.png" width="600" height="400"/>
</p>
    
Type `{}` in the template editor and choose the Save button to save the mapping template.

<p align="center">
    <img src="images/List Streams Generate Template.png" width="600" height="400"/>
</p>

## Create, describe and delete streams in Kinesis

Under the `streams` resource create a new child resource with the **Resource Name**, `{stream-name}`. After creating this your **Resources** should look like in the picture below:

<p align="center">
    <img src="images/Stream Name.png" width="600" height="200"/>
</p>

Create the following three **Methods** for `{stream-name}` resource: `POST`, `GET` and `DELETE`

### Setting up the `GET` method.

1. In the **Setup** panel you will need to select the following:,
- For **Integration type** select **AWS Service**
- For **AWS Region** choose us-east-1
- For **AWS Service** select **Kinesis**
- For **HTTP method** select `POST` 
- For **Action Type** select **User action name**
- For **Action** type `DescribeStream`
- For **Execution role** you should use the same ARN as in the previous step.

<p align="center">
    <img src="images/Describe Stream Integration.png" width="600" height="400"/>
</p>

Finally, press **Save**. This will prompt you to the **Method Execution**.

2. From here select the **Integration Request** panel. Expand the HTTP Header panel and select the following options:
- Choose **Add header** button
- Under **Name** type **Content-Type**
- Under **Mapped form** type **x-amz-json-1.1**
- Press the Create tick button

3. Expand the **Mapping Templates** panel and select the following options:
- Choose **Add mapping template** button
- Under **Cotent-Type** type **application/json**
- Press the Create tick button and choose Yes, secure this integration

4. In the template editor type the following message:

In [ ]:
{
    "StreamName": "$input.params('stream-name')"
}

Finally, choose the **Save** button to save the mapping template.


### Setting up the `POST` method.

Follow step 1 from **Setting up the GET method** section but in the **Action** section type `CreateStream`. For setting up the HTTP Header section follow step 2.

For setting up the **Mapping Templates** panel follow step 3 instruction, but add the following body mapping template in the template editor instead:

In [ ]:
{
    "ShardCount": #if($input.path('$.ShardCount') == '') 5 #else $input.path('$.ShardCount') #end,
    "StreamName": "$input.params('stream-name')"
}

In this example mapping template, the ShardCount will be fixed to a value of 5, unless the user will specify another value in the method request payload.

### Setting up the `DELETE` method.

Follow step 1 from **Setting up the GET method** section but in the **Action** section type `DeleteStream`. For setting up the HTTP Header section follow step 2.

For setting up the **Mapping Templates** panel follow step 3 instruction, but add the following body mapping template in the template editor instead:

In [ ]:
{
    "StreamName": "$input.params('stream-name')"
}

## Add records to streams in Kinesis

Under the `{stream-name}` resource create a two new child resources with the **Resource Name**, `record` and `records`. For both resources create a `PUT` method. 

### Setting up the **record** `PUT` method

Follow step 1 from **Setting up the GET method** section but in the **Action** section type `PutRecord`. For setting up the HTTP Header section follow step 2.

For setting up the **Mapping Templates** panel follow step 3 instruction, but add the following body mapping template in the template editor instead:

In [ ]:
{
    "StreamName": "$input.params('stream-name')",
    "Data": "$util.base64Encode($input.json('$.Data'))",
    "PartitionKey": "$input.path('$.PartitionKey')"
}

### Setting up the **records** `PUT` method.

Follow step 1 from **Setting up the GET method** section but in the **Action** section type `PutRecords`. For setting up the HTTP Header section follow step 2.

For setting up the **Mapping Templates** panel follow step 3 instruction, but add the following body mapping template in the template editor instead:

In [ ]:
{
    "StreamName": "$input.params('stream-name')",
    "Records": [
       #foreach($elem in $input.path('$.records'))
          {
            "Data": "$util.base64Encode($elem.data)",
            "PartitionKey": "$elem.partition-key"
          }#if($foreach.hasNext),#end
        #end
    ]
}

## API Repsonses in Python

Now that we have updated our API, we can use the Python requests library to test the new API methods and obtain a response. 
>Make sure to deploy the newest version of your API and use the correct API Invoke URL.

In [ ]:
import requests
import json

example_df = {"index": 1, "name": "Maya", "age": 25, "role": "engineer"}

# invoke url for one record, if you want to put more records replace record with records
invoke_url = "https://YourAPIInvokeURL/<YourDeploymentStage>/streams/<stream_name>/record"

#To send JSON messages you need to follow this structure
payload = json.dumps({
    "StreamName": "YourStreamName",
    "Data": {
            #Data should be send as pairs of column_name:value, with different columns separated by commas      
            "index": example_df["index"], "name": example_df["name"], "age": example_df["age"], "role": example_df["role"]
            },
            "PartitionKey": "desired-name"
            })

headers = {'Content-Type': 'application/json'}

response = requests.request("PUT", invoke_url, headers=headers, data=payload)


To see whether the request was successfully processed we can `print(response.status_code)`, which should return a status 200, indicating success.

## Visualize data coming into Kinesis Data Streams

Once you send data to a Kinesis Data Stream and receive a 200 `response.status_code`, you can visualize this data in the **Kinesis** console. 

In the console select the stream you want to look at and then choose the **Data viewer** section. Here, select the **Shard** (data will normally be stored in the first shard `shardId-000000000000`). 

In the **Starting position** section select **At timestamp**. Now you can select the **Start date**, which corresponds to the date at which you send data to your stream and the **Start time**, the time at which you started sending data (this can be an approximation). 

Alternatively, you can select **Trim horizon** as the start position, which will read all the records available in the stream if you've only posted to the stream once. If the stream has already been used, then it will read data from the last checkpoint. 

Once everything is set up, press **Get records** and you will be able to visualize the data that has been send to the stream.

<p align="center">
    <img src="images/Kinesis Records.png" width="700" height="400"/>
</p>


## Conclusion
At this point, we should have a good understanding of:

- How to create necessary permissions for Kinesis and API Gateway communication
- How to create methods that allow to list streams in Kinesis
- How to create methods that allow to create, describe and delete streams in Kinesis
- How to create methods that allow to add records to Kinesis streams